# Project: 신용카드 사기 거래 탐지 AI 경진대회

### 요약
- 이상거래 데이터와의 cosine similarity 계산
- cosine similarity가 높은 순으로 train 데이터 labeling
- Optuna 패키지를 활용하여 LGBM 모델 최적화

#### Author: 김민수(kimminsu.ds@gmail.com)

## 환경설정

### 패키지

In [3]:
import pandas as pd
import optuna
import warnings

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics          import f1_score, recall_score, precision_score
from lightgbm                 import LGBMClassifier, Dataset
from optuna.samplers          import TPESampler

In [6]:
warnings.filterwarnings(action='ignore')

### 데이터

In [7]:
train = pd.read_csv("../data/train.csv")
valid = pd.read_csv("../data/val.csv")

#### 데이터 확인

In [8]:
print("train.shape: {}".format(train.shape))
print("train.isnull().sum().sum(): {}".format(train.isnull().sum().sum()))
train.head(2)

train.shape: (113842, 31)
train.isnull().sum().sum(): 0


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,4,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972


In [9]:
print("valid.shape: {}".format(valid.shape))
print("valid.isnull().sum().sum(): {}".format(valid.isnull().sum().sum()))
valid.head(2)

valid.shape: (28462, 32)
valid.isnull().sum().sum(): 0


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,10,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,22,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0


## 전처리 

### Labeling
- train 데이터에는 label이 없음
- valid 데이터에만 label이 존재

- 0: 정상거래
- 1: 이상거래

In [10]:
print("valid.Class.value_counts():\n{}".format(valid.Class.value_counts()))

valid.Class.value_counts():
0    28432
1       30
Name: Class, dtype: int64


In [11]:
print("valid.Class.value_counts(normalize=True):\n{}".format(valid.Class.value_counts(normalize=True)))

valid.Class.value_counts(normalize=True):
0    0.998946
1    0.001054
Name: Class, dtype: float64


#### train 데이터에도 같은 비율의 이상거래가 있을 것이라 가정
- valid 데이터의 이상거래 데이터와 train 데이터의 cosine similarity를 구한 후 정렬하여 상위 0.1%의 데이터를 이상거래라고 가정

- 이상거래 데이터 저장

In [12]:
fraud = valid[valid['Class']==1].drop(columns=['ID', 'Class']).reset_index(drop=True)

- cosine similarity 계산 후 정렬

In [13]:
sim = cosine_similarity(fraud, train.drop(columns=['ID'])).transpose()
train['sim'] = list(map(sum, sim))
train['sim'] = train['sim'].apply(lambda x: round(x, 2))
train.sort_values(by='sim', ascending=False, inplace=True)
train.reset_index(drop=True, inplace=True)

- cosine similiary 기준 상위 0.1%를 이상거래로 가정

In [14]:
fraud_idx = int(train.shape[0] * 0.01)
fraud_idx

1138

In [15]:
train['Class'] = 0
train.loc[train.index.isin(range(fraud_idx)), 'Class'] = 1

In [16]:
print("train.Class.value_counts():\n{}".format(train.Class.value_counts()))

train.Class.value_counts():
0    112704
1      1138
Name: Class, dtype: int64


In [17]:
print("train.Class.value_counts(normalize=True):\n{}".format(train.Class.value_counts(normalize=True)))

train.Class.value_counts(normalize=True):
0    0.990004
1    0.009996
Name: Class, dtype: float64


#### Input data 준비

In [18]:
# Train X, y
X_train = train.drop(columns=['ID', 'sim', 'Class'])
y_train = train['Class']

# Valid X, y
X_valid = valid.drop(columns=['ID', 'Class'])
y_valid = valid['Class']

In [19]:
print("X_train.shape: {}".format(X_train.shape))
print("X_valid.shape: {}".format(X_valid.shape))
print("y_train.shape: {}".format(y_train.shape))
print("y_valid.shape: {}".format(y_valid.shape))

X_train.shape: (113842, 30)
X_valid.shape: (28462, 30)
y_train.shape: (113842,)
y_valid.shape: (28462,)


## 모델링 - LightGBM(default)

In [20]:
model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='logloss', verbose=100)

[100]	valid_0's binary_logloss: 0.0713628


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

#### 예측값 생성

In [21]:
y_valid_pred = model.predict(X_valid)
y_valid_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### 성능 측정 지표 확인

In [22]:
print("Precision: {:.4f}".format(precision_score(y_valid, y_valid_pred)))
print("Recall   : {:.4f}".format(recall_score(y_valid, y_valid_pred)))
print("F1-score : {:.4f}".format(f1_score(y_valid, y_valid_pred, average="macro")))

Precision: 0.0996
Recall   : 0.9000
F1-score : 0.5875


## 모델링 - LightGBM(Optuna)

In [23]:
sampler = TPESampler(seed=42)

def objective(trial):
    train = Dataset(X_train, label=y_train)
    valid = Dataset(X_valid, label=y_valid)
    
    param = {
        'objective' : 'binary',
        'seed'      : 42,
        'metric'    : 'binary_logloss',
        'max_depth' : trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'num_leaves'  : trial.suggest_int('num_leaves', 31, 1024),
        'colsample_bytree' : trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_uniform('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0)
    }
    
    model = LGBMClassifier(**param)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=100)
    
    y_valid_pred = model.predict(X_valid)
    return f1_score(y_valid, y_valid_pred, average="macro")

In [24]:
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=200)

[I 2022-07-09 17:21:36,530] A new study created in memory with name: no-name-e67fbe69-96a9-4b93-beb4-f1e8bc8e0b6d


[100]	valid_0's binary_logloss: 0.0119214
[200]	valid_0's binary_logloss: 0.0127557
[300]	valid_0's binary_logloss: 0.0144994
[400]	valid_0's binary_logloss: 0.0165814
[500]	valid_0's binary_logloss: 0.0188385
[600]	valid_0's binary_logloss: 0.0210508
[700]	valid_0's binary_logloss: 0.0231232
[800]	valid_0's binary_logloss: 0.0250312
[900]	valid_0's binary_logloss: 0.0267822
[1000]	valid_0's binary_logloss: 0.0283538
[1100]	valid_0's binary_logloss: 0.0297331
[1200]	valid_0's binary_logloss: 0.031038
[1300]	valid_0's binary_logloss: 0.032181
[1400]	valid_0's binary_logloss: 0.0331725


[I 2022-07-09 17:21:53,638] Trial 0 finished with value: 0.5887634713283255 and parameters: {'max_depth': 7, 'learning_rate': 0.005061576888752304, 'n_estimators': 1491, 'num_leaves': 626, 'colsample_bytree': 0.5780093202212182, 'subsample': 0.5779972601681014, 'reg_alpha': 0.2904180608409973, 'reg_lambda': 4.330880728874676}. Best is trial 0 with value: 0.5887634713283255.


[100]	valid_0's binary_logloss: 0.014209


[I 2022-07-09 17:21:55,996] Trial 1 finished with value: 0.4997363518121419 and parameters: {'max_depth': 10, 'learning_rate': 0.00017718847354806813, 'n_estimators': 139, 'num_leaves': 995, 'colsample_bytree': 0.9162213204002109, 'subsample': 0.6061695553391381, 'reg_alpha': 0.9091248360355031, 'reg_lambda': 0.9170225492671691}. Best is trial 0 with value: 0.5887634713283255.


...

[I 2022-07-09 18:01:29,086] Trial 197 finished with value: 0.4997363518121419 and parameters: {'max_depth': 3, 'learning_rate': 0.00020364416586302343, 'n_estimators': 1778, 'num_leaves': 349, 'colsample_bytree': 0.580878842506629, 'subsample': 0.5160928417279371, 'reg_alpha': 3.9554506806982843, 'reg_lambda': 0.6720173645510178}. Best is trial 75 with value: 0.7725514640071602.


[100]	valid_0's binary_logloss: 0.0136781
[200]	valid_0's binary_logloss: 0.0131377
[300]	valid_0's binary_logloss: 0.0128054
[400]	valid_0's binary_logloss: 0.0125768
[500]	valid_0's binary_logloss: 0.012407
[600]	valid_0's binary_logloss: 0.0122832
[700]	valid_0's binary_logloss: 0.0121864
[800]	valid_0's binary_logloss: 0.0121232
[900]	valid_0's binary_logloss: 0.0120777
[1000]	valid_0's binary_logloss: 0.0120502
[1100]	valid_0's binary_logloss: 0.0120379
[1200]	valid_0's binary_logloss: 0.0120391
[1300]	valid_0's binary_logloss: 0.0120519
[1400]	valid_0's binary_logloss: 0.0120759
[1500]	valid_0's binary_logloss: 0.0121104
[1600]	valid_0's binary_logloss: 0.0121559
[1700]	valid_0's binary_logloss: 0.0122096
[1800]	valid_0's binary_logloss: 0.0122704


[I 2022-07-09 18:01:57,958] Trial 198 finished with value: 0.6250366692036734 and parameters: {'max_depth': 4, 'learning_rate': 0.00039925619105127724, 'n_estimators': 1820, 'num_leaves': 304, 'colsample_bytree': 0.5922348804271866, 'subsample': 0.5219616453492724, 'reg_alpha': 3.7335309368633425, 'reg_lambda': 0.49037675964964533}. Best is trial 75 with value: 0.7725514640071602.


[100]	valid_0's binary_logloss: 0.0139437
[200]	valid_0's binary_logloss: 0.013448
[300]	valid_0's binary_logloss: 0.0131182
[400]	valid_0's binary_logloss: 0.0128855
[500]	valid_0's binary_logloss: 0.0127086
[600]	valid_0's binary_logloss: 0.0125741
[700]	valid_0's binary_logloss: 0.012462
[800]	valid_0's binary_logloss: 0.0123762
[900]	valid_0's binary_logloss: 0.0123043
[1000]	valid_0's binary_logloss: 0.0122487
[1100]	valid_0's binary_logloss: 0.0122017
[1200]	valid_0's binary_logloss: 0.0121656
[1300]	valid_0's binary_logloss: 0.0121395
[1400]	valid_0's binary_logloss: 0.0121226
[1500]	valid_0's binary_logloss: 0.0121151
[1600]	valid_0's binary_logloss: 0.0121154
[1700]	valid_0's binary_logloss: 0.0121217


[I 2022-07-09 18:02:09,133] Trial 199 finished with value: 0.4997363518121419 and parameters: {'max_depth': 3, 'learning_rate': 0.00029592302746682603, 'n_estimators': 1710, 'num_leaves': 281, 'colsample_bytree': 0.5660146865616346, 'subsample': 0.5323223844493876, 'reg_alpha': 4.1631036790790565, 'reg_lambda': 0.6376520844004635}. Best is trial 75 with value: 0.7725514640071602.


In [26]:
best_model = study.best_trial
best_params = best_model.params

print("Best model:")
print("  F1-score: {}".format(round(best_model.value, 4)))
print("  params  : {}".format(best_params))

Best model:
  F1-score: 0.7726
  params  : {'max_depth': 3, 'learning_rate': 0.0005320502362980809, 'n_estimators': 1202, 'num_leaves': 409, 'colsample_bytree': 0.6822670702134966, 'subsample': 0.6968505503703223, 'reg_alpha': 1.7758721383692797, 'reg_lambda': 0.2537070327857336}


In [27]:
model = LGBMClassifier(**best_params)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='logloss', verbose=100)

[100]	valid_0's binary_logloss: 0.0134457
[200]	valid_0's binary_logloss: 0.0129045
[300]	valid_0's binary_logloss: 0.0125956
[400]	valid_0's binary_logloss: 0.0124055
[500]	valid_0's binary_logloss: 0.0122773
[600]	valid_0's binary_logloss: 0.0121883
[700]	valid_0's binary_logloss: 0.0121355
[800]	valid_0's binary_logloss: 0.0121129
[900]	valid_0's binary_logloss: 0.0121142
[1000]	valid_0's binary_logloss: 0.012135
[1100]	valid_0's binary_logloss: 0.0121756
[1200]	valid_0's binary_logloss: 0.0122324


LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.6822670702134966, importance_type='split',
               learning_rate=0.0005320502362980809, max_depth=3,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1202, n_jobs=-1, num_leaves=409, objective=None,
               random_state=None, reg_alpha=1.7758721383692797,
               reg_lambda=0.2537070327857336, silent='warn',
               subsample=0.6968505503703223, subsample_for_bin=200000,
               subsample_freq=0)

In [28]:
y_valid_pred = model.predict(X_valid)
y_valid_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
print("Precision: {:.4f}".format(precision_score(y_valid, y_valid_pred)))
print("Recall   : {:.4f}".format(recall_score(y_valid, y_valid_pred)))
print("F1-score : {:.4f}".format(f1_score(y_valid, y_valid_pred, average="macro")))

Precision: 0.4333
Recall   : 0.4333
F1-score : 0.7164


## 최종 결과 생성

#### 예측값 생성

In [30]:
test = pd.read_csv("../data/test.csv")
X_test = test.drop(columns=['ID'])
y_test_pred = model.predict(X_test)

#### 제출 파일 저장

In [31]:
submit = pd.read_csv("../data/sample_submission.csv")
submit['Class'] = y_test_pred
submit.to_csv("./submit.csv", index=False)